
# Policy, Claims & Telematics DLT Ingestion pipeline

The raw data is saved within our Unity Catalog Volume. Raw files are incrementally ingested and save them as a Delta Table using  `cloud_files`. 
<b> Policy, Claims & Telematics </b> which are in different formats, and come from different sources and are consolidated in the claims lakehouse. 

<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=lakehouse&org_id=4325389025886535&notebook=%2F01-Data-Ingestion%2F01.1-DLT-Ingest-Policy-Claims&demo_name=lakehouse-fsi-smart-claims&event=VIEW&path=%2F_dbdemos%2Flakehouse%2Flakehouse-fsi-smart-claims%2F01-Data-Ingestion%2F01.1-DLT-Ingest-Policy-Claims&version=1">


This demo started for you a <a dbdemos-pipeline-id="dlt-fsi-smart-claims" href="#joblist/pipelines/0637ecdc-72fe-46fa-a8a1-14453616fc77" target="_blank">DLT Pipeline</a> using this notebook! Explore it to see its execution out of the box.


## Raw Data Ingestion

In [0]:
import dlt
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, concat, col
from pyspark.sql import types as T

catalog = "main"
db = "dbdemos_fsi_smart_claims"
volume_name = "volume_claims"

In [0]:
@dlt.table(comment="The raw claims data loaded from json files.")
def raw_claim():
  return (
    spark.readStream.format("cloudFiles")
          .option("cloudFiles.format", "json")
          .option("cloudFiles.inferColumnTypes", "true")
          .load(f"/Volumes/{catalog}/{db}/{volume_name}/Claims"))

In [0]:
@dlt.table(comment="Policy data loaded from csv files.")
def raw_policy():
    return (
      spark.readStream.format("cloudFiles")
            .option("cloudFiles.format", "csv")
            .option("cloudFiles.schemaHints", "ZIPCODE int")
            .option("cloudFiles.inferColumnTypes", "true")
            .load(f"/Volumes/{catalog}/{db}/{volume_name}/Policies"))

In [0]:
@dlt.table(comment="Load Telematics (IoT) streaming data")
def raw_telematics():
  return (
    spark.readStream.format("cloudFiles")
          .option("cloudFiles.format", "parquet")
          .load(f"/Volumes/{catalog}/{db}/{volume_name}/Telematics"))


## Data Cleaning, Transormation data and tables

In [0]:
def flatten_struct(df):
    for field in df.schema.fields:
        if isinstance(field.dataType, T.StructType):
            for child in field.dataType:
                df = df.withColumn(field.name + '_' + child.name, F.col(field.name + '.' + child.name))
            df = df.drop(field.name)
    return df

In [0]:
@dlt.table(comment="Average telematics")
def telematics():
  return (dlt.read('raw_telematics').groupBy("chassis_no").agg(
                F.avg("speed").alias("telematics_speed"),
                F.avg("latitude").alias("telematics_latitude"),
                F.avg("longitude").alias("telematics_longitude")))
                
@dlt.table
@dlt.expect_all({"valid_policy_number": "policy_no IS NOT NULL"})
def policy():
    # Read the staged policy records into memory
    return (dlt.readStream("raw_policy")
                .withColumn("premium", F.abs(col("premium")))
                # Reformat the incident date values
                .withColumn("pol_eff_date", F.to_date(col("pol_eff_date"), "dd-MM-yyyy"))
                .withColumn("pol_expiry_date", F.to_date(col("pol_expiry_date"), "dd-MM-yyyy"))
                .withColumn("pol_issue_date", F.to_date(col("pol_issue_date"), "dd-MM-yyyy"))
                .withColumn("address", concat(col("BOROUGH"), lit(", "), col("ZIP_CODE").cast("string")))
                .drop('_rescued_data'))

@dlt.table
@dlt.expect_all({"valid_claim_number": "claim_no IS NOT NULL"})
def claim():
    # Read the staged claim records into memory
    claim = dlt.readStream("raw_claim")
    claim = flatten_struct(claim)  
    
    # Update the format of all date/time features
    return (claim.withColumn("claim_date", F.to_date(F.col("claim_date")))
                 .withColumn("incident_date", F.to_date(F.col("incident_date"), "yyyy-MM-dd"))
                 .withColumn("driver_license_issue_date", F.to_date(F.col("driver_license_issue_date"), "dd-MM-yyyy"))
                 .drop('_rescued_data'))


## Joining Policy and Claim into a single table single table.

In [0]:
@dlt.table(comment = "Curated claim joined with policy records")
def claim_policy():
    # Read the staged policy records
    policy = dlt.read("policy")
    # Read the staged claim records
    claim = dlt.readStream("claim")
    
    return claim.join(policy, on="policy_no")